# Implementation of normalization class

### Background
To improve data normalization in MAELSTROM, a dedicated class for this job is introduced. <br>
However, the implementation reveals some low-level bugs with xarray that are documented here.

### Problem statement
Let's start with the *errornous* approach in which we allow handling of datasets in the class. Note that this approach should work according to [xarray's documentation](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.std.html). It also works technically (i.e. it does not throw an error), but it produces strange results due to unrealistic values from the `std`-method on the dataset.

The (verbose) source-code looks as follows:

In [2]:
import os

from abc import ABC, abstractmethod
from typing import Union, List
import xarray as xr
import numpy as np

da_or_ds = Union[xr.DataArray, xr.Dataset]

class Normalize(ABC):
    """
    Abstract class for normalizing data.
    """
    def __init__(self, method: str, norm_dims: List):
        self.method = method
        self.norm_dims = norm_dims
        self.norm_stats = None

    def normalize(self, data: xr.DataArray, **stats):
        
        
        norm_stats = self.get_required_stats(data, **stats)
        data_norm = self.normalize_data(data, *norm_stats)

        return data_norm

    def denormalize(self, data: da_or_ds, **stats):
        norm_stats = self.get_required_stats(data, *stats)
        data_denorm = self.denormalize_data(data, *norm_stats)

        return data_denorm

    @abstractmethod
    def get_required_stats(self, data, **stats):
        """
        Function to retrieve either normalization parameters from data or from keyword arguments
        """
        pass

    @abstractmethod
    def normalize_data(data, *norm_param):
        """
        Function to normalize data.
        """
        pass

    @abstractmethod
    def denormalize_data(data, *norm_param):
        """
        Function to denormalize data.
        """
        pass

The child class for z-score normalization looks then as follows:

In [3]:
class ZScore(Normalize):

    def __init__(self, norm_dims: List):
        super().__init__("z_score", norm_dims)
        self.norm_stats = {"mu": None, "sigma": None}

    def get_required_stats(self, data: da_or_ds, **stats):

        mu, std = stats.get("mu", self.norm_stats["mu"]), stats.get("sigma", self.norm_stats["sigma"])

        if mu is None or std is None:
            print("Retrieve mu and sigma from data...")
            mu, std = data.mean(self.norm_dims), data.std(self.norm_dims)
            self.norm_stats = {"mu": mu, "sigma": std}
            print(self.norm_stats)
        else:
            print("Mu and sigma are parsed for (de-)normalization.")
            print(mu)
            print(std)
            
        return mu, std
            
    @staticmethod     
    def normalize_data(data, mu, std):
        data_norm = (data - mu) / std
        
        return data_norm
    
    @staticmethod     
    def denormalize_data(data, mu, std):
        data_denorm = data * std + mu
        
        return data_denorm
    

We load the validation data of the downscaling Tier-2 dataset for testing purposes.

In [4]:
datadir = "/p/project/deepacf/maelstrom/data/ap5/tier2"
datafile = os.path.join(datadir, "maelstrom-downscaling-tier2_test.nc")

ds = xr.open_dataset(datafile)

Then we apply the z-score normalization on the dataset:

In [6]:
norm_dims = ["time", "rlat", "rlon"]
zscore_norm = ZScore(norm_dims)

ds_norm1 = zscore_norm.normalize(ds)

Retrieve mu and sigma from data...
{'mu': <xarray.Dataset>
Dimensions:       ()
Data variables:
    rotated_pole  float64 1.0
    2t_in         float32 282.8
    sshf_in       float32 -7.078e+04
    slhf_in       float32 -1.565e+05
    blh_in        float32 529.6
    10u_in        float32 0.1829
    10v_in        float32 0.1448
    z_in          float32 5.686e+03
    t850_in       float32 278.6
    t925_in       float32 282.7
    hsurf_tar     float32 571.6
    t_2m_tar      float32 283.2, 'sigma': <xarray.Dataset>
Dimensions:       ()
Data variables:
    rotated_pole  float64 0.0
    2t_in         float64 104.5
    sshf_in       float64 1.973e+05
    slhf_in       float64 2.177e+05
    blh_in        float64 486.9
    10u_in        float64 2.089
    10v_in        float64 1.616
    z_in          float64 4.809e+03
    t850_in       float64 104.5
    t925_in       float64 104.5
    hsurf_tar     float64 511.0
    t_2m_tar      float64 104.5}


As we can see, the calculated standard deviation of the temperature variables (and others?) yields unexpected results which in would result into incorrect normalization. To prove this statement, we calculate the standard deviation manually:

In [7]:
t2m = ds["2t_in"]
sigma_t2m = np.sqrt(((t2m - t2m.mean(dim=norm_dims))**2).mean(dim=norm_dims))
print(sigma_t2m)              # note that omitting the parameter-setting dim=norm_dims would give the same result

<xarray.DataArray '2t_in' ()>
array(9.052816, dtype=float32)


By contrast, conversion of the dataset to a xr.DataArray, where the variables are assigned to a new dimension, gives the correct result.

In [8]:
da = ds.to_array(dim="variables")

zscore_norm2 = ZScore(norm_dims)     # get fresh class instance to avoid precomputed normalization parameters
da_norm2 = zscore_norm2.normalize(da)

Retrieve mu and sigma from data...
{'mu': <xarray.DataArray (variables: 12)>
array([ 1.00000000e+00,  2.82832158e+02, -7.07827281e+04, -1.56503764e+05,
        5.29617935e+02,  1.82885170e-01,  1.44832186e-01,  5.68644335e+03,
        2.78623048e+02,  2.82670052e+02,  5.71572380e+02,  2.83245770e+02])
Coordinates:
  * variables  (variables) <U12 'rotated_pole' '2t_in' ... 't_2m_tar', 'sigma': <xarray.DataArray (variables: 12)>
array([0.00000000e+00, 9.05282455e+00, 2.24066973e+05, 2.36301865e+05,
       5.11373825e+02, 2.43462441e+00, 1.87755217e+00, 4.48457913e+03,
       7.45010479e+00, 8.21746109e+00, 4.97467674e+02, 9.03267566e+00])
Coordinates:
  * variables  (variables) <U12 'rotated_pole' '2t_in' ... 't_2m_tar'}


Let's perform some further tests.

In [9]:
vardict = {"variables": "2t_in"}
t2m_norm1 = da_norm2.sel(vardict)
mu, std = zscore_norm2.norm_stats["mu"].sel(vardict), zscore_norm2.norm_stats["sigma"].sel(vardict)

# normalize 2t_in with precomputed normalization parameters
t2m_norm2 = zscore_norm2.normalize(t2m, mu=mu.values, sigma=std.values)
# reset norm_stats manually...
zscore_norm2.norm_stats = {"mu": None, "sigma": None}
# ... to trigger computation
t2m_norm3 = zscore_norm2.normalize(t2m)

Mu and sigma are parsed for (de-)normalization.
282.8321579222078
9.052824545594893
Retrieve mu and sigma from data...
{'mu': <xarray.DataArray '2t_in' ()>
array(282.83246, dtype=float32), 'sigma': <xarray.DataArray '2t_in' ()>
array(9.052816, dtype=float32)}


In [10]:
assert np.all(np.isclose(t2m_norm1, t2m_norm2, atol=1.e-04)), "t2m_norm1 and t2m_norm2 differ!"
assert np.all(np.isclose(t2m_norm2, t2m_norm3, atol=1.e-04)), "t2m_norm2 and t2m_norm3 differ!"
assert np.all(np.isclose(ds_norm1["2t_in"], t2m_norm2, atol=1.e-04)), "ds_norm1 and t2m_norm2 differ!"

AssertionError: ds_norm1 and t2m_norm2 differ!

As expected, the normalized values differ substanially due to the errornous value for the standard deviataion.
We can further trace the error which even happens when we apply the `std`-method on a Data Array with default-behaviur (i.e. without setting `dim`).

In [31]:
print(norm_dims)
print(ds["2t_in"].coords)

['time', 'rlat', 'rlon']
Coordinates:
  * time     (time) datetime64[ns] 2018-01-01T01:00:00 ... 2018-12-31T23:00:00
  * rlon     (rlon) float64 -8.273 -8.218 -8.163 -8.108 ... -1.838 -1.783 -1.728
  * rlat     (rlat) float64 -3.933 -3.878 -3.823 -3.768 ... 1.182 1.237 1.292


In [32]:
print(ds.std(skipna=True))                     # as above
print(ds["2t_in"].std(skipna=True))            # same error when retrievin 2t_in from the dataset only and defaulting .std()
print(ds["2t_in"].std(norm_dims, skipna=True))   # setting dimensions for .std()

<xarray.Dataset>
Dimensions:       ()
Data variables:
    rotated_pole  float64 0.0
    2t_in         float64 104.5
    sshf_in       float64 1.973e+05
    slhf_in       float64 2.177e+05
    blh_in        float64 486.9
    10u_in        float64 2.089
    10v_in        float64 1.616
    z_in          float64 4.809e+03
    t850_in       float64 104.5
    t925_in       float64 104.5
    hsurf_tar     float64 511.0
    t_2m_tar      float64 104.5
<xarray.DataArray '2t_in' ()>
array(104.4526062)
<xarray.DataArray '2t_in' ()>
array(9.052816, dtype=float32)


### The workaround

In the following, we ensure that the normalization class does not work on `xr.Datasets`. Note furthermore, that `norm_dims=None` also fails since the default approach may run into the same issues:

In [120]:
class Normalize(ABC):
    """
    Abstract class for normalizing data.
    """
    def __init__(self, method: str, norm_dims: List):
        self.method = method
        self.norm_dims = norm_dims
        self.norm_stats = None

    def normalize(self, data: xr.DataArray, **stats):
        """
        Normalize data.
        :param data: The DataArray to be normalized.
        :param **stats: Parameters to perform normalization. Must fit to normalization type!
        :return: DataArray with normalized data.
        """
        # sanity checks
        if not isinstance(data, xr.DataArray):
            raise TypeError(f"Passed data must be a xarray.DataArray, but is of type {str(type(data))}.")
            
        _ = self._check_norm_dims(data)
        # do the computation            
        norm_stats = self.get_required_stats(data, **stats)
        data_norm = self.normalize_data(data, *norm_stats)

        return data_norm

    def denormalize(self, data: da_or_ds, **stats):
        """
        Denormalize data.
        :param data: The DataArray to be denormalized.
        :param **stats: Parameters to perform denormalization. Must fit to normalization type!
        :return: DataArray with denormalized data.
        """
        # sanity checks
        if not isinstance(data, xr.DataArray):
            raise TypeError(f"Passed data must be a xarray.DataArray, but is of type {str(type(data))}.")
            
        _ = self._check_norm_dims(data)
        # do the computation       
        norm_stats = self.get_required_stats(data, *stats)
        data_denorm = self.denormalize_data(data, *norm_stats)

        return data_denorm
    
    @property
    def norm_dims(self):
        return self._norm_dims
    
    @norm_dims.setter
    def norm_dims(self, norm_dims):
        if norm_dims is None:
            raise AttributeError("norm_dims must not be None. Please parse a list of dimensions" +
                                 "over which normalization should be applied.")
        
        self._norm_dims = list(norm_dims)
        
    def _check_norm_dims(self, data):
        """
        Check if dimension for normalization reside in dimensions of data.
        :param data: the data (xr.DataArray) to be normalized
        :return True: in case of passed check, a ValueError is risen else
        """
        data_dims = list(data.dims)
        norm_dims_check = [norm_dim in data_dims for norm_dim in self.norm_dims]
        if not all(norm_dims_check):
            imiss = np.where(~np.array(norm_dims_check))[0]
            miss_dims = list(np.array(self.norm_dims)[imiss])
            raise ValueError("The following dimensions do not reside in the data: " +
                             f"{', '.join(miss_dims)}")

        return True
    
    def save_norm_to_file(self, js_file):
        """
        Write normalization parameters to file.
        :param js_file: Path to JSON-file to be created.
        :return: -
        """
        if self.norm_stats is None:
            raise AttributeError("norm_stats is still None. Please run (de-)normalization to get parameters.")
        
        if any([stat is None for stat in self.norm_stats.values()]):
            raise AttributeError("Some parameters of norm_stats are None.")
            
        norm_serialized = {key: da.to_dict() for key, da in norm_dict.items()}
        
        with open(js_file, "w") as jsf:
            js.dump(norm_dict_serialized, jsf)
        
    def read_norm_from_file(self, js_file):
        """
        Read normalization parameters from file. Inverse function to write_norm_from_file.
        :param js_file: Path to JSON-file to be read.
        :return: Parameters set to self.norm_stats
        """
        with open(js_file, "r") as jsf:
            norm_data = js.load(jsf)
            
        norm_dict_restored = {key: xr.DataArray.from_dict(da_dict) for key, da_dict in norm_data.items()}
        
        self.norm_stats = norm_dict_restored    
        

    @abstractmethod
    def get_required_stats(self, data, **stats):
        """
        Function to retrieve either normalization parameters from data or from keyword arguments
        """
        pass

    @abstractmethod
    def normalize_data(data, *norm_param):
        """
        Function to normalize data.
        """
        pass

    @abstractmethod
    def denormalize_data(data, *norm_param):
        """
        Function to denormalize data.
        """
        pass
    
class ZScore(Normalize):

    def __init__(self, norm_dims: List):
        super().__init__("z_score", norm_dims)
        self.norm_stats = {"mu": None, "sigma": None}

    def get_required_stats(self, data: da_or_ds, **stats):
        """
        Get required parameters for z-score normalization. They are either computed from the data 
        or can be parsed as keyword arguments.
        :param data: the data to be (de-)normalized
        :param mu: keyword argument for mean used for normalization
        :param sigma: keyword argument for standard deviation for normalization
        :return (mu, sigma): Parameters for normalization
        """
        mu, std = stats.get("mu", self.norm_stats["mu"]), stats.get("sigma", self.norm_stats["sigma"])

        if mu is None or std is None:
            print("Retrieve mu and sigma from data...")
            mu, std = data.mean(self.norm_dims), data.std(self.norm_dims)
            self.norm_stats = {"mu": mu, "sigma": std}
            print(self.norm_stats)
        else:
            print("Mu and sigma are parsed for (de-)normalization.")
            print(mu)
            print(std)
            
        return mu, std
            
    @staticmethod     
    def normalize_data(data, mu, std):
        """
        Perform z-score normalization on data
        :param data: Data array of interest
        :param mu: mean of data for normalization
        :param std: standard deviation of data for normalization
        :return data_norm: normalized data
        """
        data_norm = (data - mu) / std
        
        return data_norm
    
    @staticmethod     
    def denormalize_data(data, mu, std):
        """
        Perform z-score denormalization on data.
        :param data: Data array of interest
        :param mu: mean of data for denormalization
        :param std: standard deviation of data for denormalization
        :return data_norm: denormalized data
        """
        data_denorm = data * std + mu
        
        return data_denorm

To enable data processing, we start by converting the dataset to a data array.

In [110]:
da = ds.to_array(dim="variables")

Let's perform various tests:

In [121]:
zscore_norm = ZScore(["rlat", "rlon", "time"])

# normalize the resulting Data Array...
da_norm = zscore_norm.normalize(da)

Retrieve mu and sigma from data...
{'mu': <xarray.DataArray (variables: 12)>
array([ 1.00000000e+00,  2.82832158e+02, -7.07827281e+04, -1.56503764e+05,
        5.29617935e+02,  1.82885170e-01,  1.44832186e-01,  5.68644335e+03,
        2.78623048e+02,  2.82670052e+02,  5.71572380e+02,  2.83245770e+02])
Coordinates:
  * variables  (variables) <U12 'rotated_pole' '2t_in' ... 't_2m_tar', 'sigma': <xarray.DataArray (variables: 12)>
array([0.00000000e+00, 9.05282455e+00, 2.24066973e+05, 2.36301865e+05,
       5.11373825e+02, 2.43462441e+00, 1.87755217e+00, 4.48457913e+03,
       7.45010479e+00, 8.21746109e+00, 4.97467674e+02, 9.03267566e+00])
Coordinates:
  * variables  (variables) <U12 'rotated_pole' '2t_in' ... 't_2m_tar'}


In [124]:
# ... and compare:
assert np.all(np.isclose(da_norm.sel({"variables": "2t_in"}), t2m_norm1, atol=1.e-06)), "da_norm and t2m_norm1 differ!"
assert np.all(np.isclose(t2m_norm1, t2m_norm2, atol=1.e-04)), "t2m_norm1 and t2m_norm2 differ!"

Except for spurious deviations due to changes in the normalization parameters (why ever this happens?!), the results now coincide. At least, errors smaller than 1.e-04 can also be neglected in the normalized space for our purposes which aims to project the values into a data range suitable for backpropgation in a neural network.

In the following, we perform some further tests:

In [126]:
js_file = "./test.json"

# save normalization parameters to file
zscore_norm.save_norm_to_file(js_file)

# instantiate fresh instance and get normalization parameters from file
zscore_norm_new = ZScore(["rlat", "rlon", "time"])
zscore_norm_new.read_norm_from_file(js_file)

# apply normalization without retrieval from data (see print-statement!)
da_norm = zscore_norm_new.normalize(da)

Mu and sigma are parsed for (de-)normalization.
<xarray.DataArray (variables: 12)>
array([ 1.00000000e+00,  2.82832158e+02, -7.07827281e+04, -1.56503764e+05,
        5.29617935e+02,  1.82885170e-01,  1.44832186e-01,  5.68644335e+03,
        2.78623048e+02,  2.82670052e+02,  5.71572380e+02,  2.83245770e+02])
Coordinates:
  * variables  (variables) <U12 'rotated_pole' '2t_in' ... 't_2m_tar'
<xarray.DataArray (variables: 12)>
array([0.00000000e+00, 9.05282455e+00, 2.24066973e+05, 2.36301865e+05,
       5.11373825e+02, 2.43462441e+00, 1.87755217e+00, 4.48457913e+03,
       7.45010479e+00, 8.21746109e+00, 4.97467674e+02, 9.03267566e+00])
Coordinates:
  * variables  (variables) <U12 'rotated_pole' '2t_in' ... 't_2m_tar'


Finally check, if we still obtain the expected result:

In [128]:
assert np.all(np.isclose(da_norm.sel({"variables": "2t_in"}), t2m_norm1, atol=1.e-06)), "da_norm and t2m_norm1 differ!"